# Import

In [1]:
import pandas as pd
import requests
import json
import os
import sys
import time 
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import base64

# Get token

In [2]:
# Dictionary to store multiple client credentials
CURRENT_CLIENT_ID = 1
CLIENT_CREDENTIALS = {
    1 : {
        'client_id': '489161f1d161402ca00a10e470b9b808',
        'client_secret': '6041044fc5dc498480994298ed0163c8'
    },
    2: {
        'client_id': 'dc4c9136605a478e9c836c59691cd474', 
        'client_secret': '7dca333866ab4a05be49bd4078d98744'
    },
    3: {
        'client_id': 'b6b19e48101040f5943ce1050a04b153', 
        'client_secret': '9182c41a5f4746d980130a3270823557'
    },
    4: {
        'client_id': 'cf1516d021e2489daf208a4f973544d2', 
        'client_secret': 'ebfc8b2428b94aa8977b0b34ad9a518f'
    },
    5: {
        'client_id': '106f35d5e02347749a5d11967e00a5f2', 
        'client_secret': 'fc971fb3df9b4710b4b73cffb4172a85'
    },
    6: {
        'client_id': 'eeda886daea447de937b498eeab2ca24', 
        'client_secret': '2f62c397322d4dcc8efdf53a2e0db85c'
    },
    7: {
        'client_id': '464d8814654d437683582a5c189a6e82', 
        'client_secret': 'aed59a52534a404597f37d735acbecfb'
    },
    8: {
        'client_id': '9f89459be490410a89ea56ee73caa09d', 
        'client_secret': 'a96d44704bcc4f879c446125b0963395'
    },
    9: {
        'client_id': 'f456115fa6df45a3bc04469a5a4d4384', 
        'client_secret': 'caec468cc4ca40bcbf6012c2ba4afea4'
    },
    10: {
        'client_id': '97961af1175d4bc28ba5f9d1aae89598', 
        'client_secret': 'a045e19fe1d24e549a7fdc9344f37ffe'
    }
}

In [3]:
def get_token(client_id, client_secret):
    # Mã hóa client_id và client_secret thành base64
    auth_string = f"{client_id}:{client_secret}"
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = base64.b64encode(auth_bytes).decode('utf-8')

    # Thiết lập thông tin yêu cầu
    auth_url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': f'Basic {auth_base64}'
    }
    data = {
        'grant_type': 'client_credentials'
    }

    # Gửi yêu cầu POST để lấy token
    response = requests.post(auth_url, headers=headers, data=data)

    # Kiểm tra phản hồi và trả về token nếu thành công
    if response.status_code == 200:
        token = response.json().get('access_token')
        return token
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def get_valid_token(client_id, client_secret):
    token = None
    while token is None:
        token = get_token(client_id, client_secret)
        if token is None:
            print("Token expired or error. Retrying in 1 minute...")
            time.sleep(60)  # Nếu token không hợp lệ, chờ 1 phút và thử lại
    return token


In [4]:
def new_token(id):
    try:
        credentials = CLIENT_CREDENTIALS[id]
        client_id = credentials['client_id']
        client_secret = credentials['client_secret']
        
        return get_valid_token(client_id, client_secret)
        
    except KeyError:
        print(f"Error id: {id}")
        raise KeyError(f"No credentials found for app: {id}")
    except requests.exceptions.RequestException as e:
        raise Exception(f"Failed to get token: {str(e)}")

In [5]:
token = new_token(CURRENT_CLIENT_ID)
print(token)

BQDbXaqSFQ292Oj0ZXJLG5u8M8lU6OMpaZHUWHeWCNqLiumSpg6H0qrIcXF0zDX10L_l-j9pu_khXU1LTgJoInb5WjH6wpm83FbzUHaihL6kRGn-GXg


# Get popularity
Từ file `Old_Data_Colecting`, ta đã thu thập được một số thông tin cho các bài hát.
Vì [endpoint](https://developer.spotify.com/documentation/web-api/reference/get-audio-features) của spotify hiện không hoạt động. Ta đành dùng dữ liệu cũ đã thu thập được và tiến hành thêm một số thuộc tính.
- popularity
- release day

In [6]:
class RateLimitExceededException(Exception):
    """Exception raised when Spotify API returns a 429 status code."""
    pass

# Hàm fetch_data
def fetch_data(id_chunk, token):
    global CURRENT_CLIENT_ID
    url = f"https://api.spotify.com/v1/tracks?ids={id_chunk}"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        # Lấy cả popularity và release_date
        return {
            track['id']: {
                'popularity': track['popularity'],
                'release_date': track['album']['release_date']
            }
            for track in data['tracks'] if 'popularity' in track and 'album' in track
        }

    elif response.status_code == 429:
        # Throw custom exception
        raise RateLimitExceededException("Rate limit exceeded. Please try again later.")
    
    else:
        raise Exception(f"Error fetching data from Spotify API: {response.text}")

# Hàm xử lý chunk và thêm popularity + release_date
def process_chunk(chunk, token):
    ids = chunk['Track ID'].tolist()
    id_chunks = [",".join(ids[i:i+50]) for i in range(0, len(ids), 50)]
    data = {}
    for id_chunk in id_chunks:
        data.update(fetch_data(id_chunk, token))
    
    # Thêm cột popularity và release_date vào chunk
    chunk['popularity'] = chunk['Track ID'].map(lambda x: data.get(x, {}).get('popularity', None))
    chunk['release_date'] = chunk['Track ID'].map(lambda x: data.get(x, {}).get('release_date', None))
    return chunk

In [7]:
def save_progress(filtered_songs, output_csv):
    """Save current progress to CSV"""
    if filtered_songs:
        pd.concat(filtered_songs).to_csv(output_csv, index=False)
        print(f"Progress saved to {output_csv}")

In [8]:
def main(input_csv, output_csv, target_count = 10e9):
    global CURRENT_CLIENT_ID
    filtered_songs = []
    token = new_token(CURRENT_CLIENT_ID)
    
    try:
        successfull = 1
        for chunk in pd.read_csv(input_csv, chunksize=50):
            try:
                # If token is invalid, get new one
                if not token:
                    token = new_token(CURRENT_CLIENT_ID)
                    if not token:
                        raise Exception("Failed to get valid token")
                
                chunk = process_chunk(chunk, token)
                filtered_songs.append(chunk)
                print(f"{successfull} chunk append sucessfully, id = {CURRENT_CLIENT_ID}")
                successfull += 1
                
                if sum(len(f) for f in filtered_songs) >= target_count:
                    break
                    
            except RateLimitExceededException:
                print("Rate limit reached. Switching client ID and retrying...")
                save_progress(filtered_songs, output_csv)
                CURRENT_CLIENT_ID += 1
                token = new_token(CURRENT_CLIENT_ID)
                continue
                
            except requests.exceptions.HTTPError as e:
                raise  # Re-raise other HTTP errors
                
    except Exception as e:
        print(f"Error occurred: {e}")
        save_progress(filtered_songs, output_csv)
        return
        
    # Final save
    result_df = pd.concat(filtered_songs).head(target_count)
    result_df.to_csv(output_csv, index=False)
    print(f"Completed! Saved {len(result_df)} songs to {output_csv}")

In [ ]:
main("/kaggle/input/music-dataset/merged_tracks.csv", "/kaggle/working/music_data.csv")

1 chunk append sucessfully, id = 1
Rate limit reached. Switching client ID and retrying...
Progress saved to /kaggle/working/music_data.csv
Rate limit reached. Switching client ID and retrying...
Progress saved to /kaggle/working/music_data.csv
Rate limit reached. Switching client ID and retrying...
Progress saved to /kaggle/working/music_data.csv
2 chunk append sucessfully, id = 4
3 chunk append sucessfully, id = 4
4 chunk append sucessfully, id = 4
5 chunk append sucessfully, id = 4
6 chunk append sucessfully, id = 4
7 chunk append sucessfully, id = 4
8 chunk append sucessfully, id = 4
9 chunk append sucessfully, id = 4
10 chunk append sucessfully, id = 4
11 chunk append sucessfully, id = 4
12 chunk append sucessfully, id = 4
13 chunk append sucessfully, id = 4
14 chunk append sucessfully, id = 4
15 chunk append sucessfully, id = 4
16 chunk append sucessfully, id = 4
17 chunk append sucessfully, id = 4
18 chunk append sucessfully, id = 4
19 chunk append sucessfully, id = 4
20 chunk a